# アカウントのテキストと画像をもらう

In [ ]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token)
        self.count = 5


    @property
    def text_media_get(self):
        """_summary_
        usernameにアカウント名を入力

        画像を取る場合は以下を有効にする
        for tweets_img in target_tweets.includes['media']:

        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username='cutethigh') # usernameにアカウント名を入力
        target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets')

        while True:
            print(target_tweets)
            next_token = target_tweets.meta['next_token']

            # テキストを取る
            for tweets in target_tweets.data:
                text = tweets.data['text']
                print(text)

            # 画像を取る
            for tweets_img in target_tweets.includes['media']:
                img_url = tweets_img['url']
                print(img_url)

            tweet['tweet'].append({"text": text,"img": img_url})
            print(tweet)

            with open('/spanking.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets', pagination_token=next_token)

            if next_token is None:
                break



t = Tweet_text()
t.text_media_get


        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


# v1でテキストと画像取得できた

- v2 テキストと画像取得しようとしたけどうまくいかない→ v1.1でやった方が楽という結論
- v1でOK

In [71]:
import tweepy
import API_config_katudon
import json
import requests
import re
import os


class Tweet_text:

    def __init__(self, username):

        auth = tweepy.OAuthHandler(API_config_katudon.API_KEY, API_config_katudon.API_SECRET)
        auth.set_access_token(API_config_katudon.ACCESS_TOKEN, API_config_katudon.ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.__username = username

    def text_image_get(self):


        count_no = 200
        tweet: dict[dict] = {}
        tweet['tweet'] = []

        results: iter = tweepy.Cursor(self.api.user_timeline, # タイムラインの取得
                    id=self.__username, # 取得対象のユーザーを指定
                    include_entities=True, # 省略されたリンクを全て取得
                    tweet_mode='extended', # 省略されたツイートを全て取得
                    lang='ja',
                    exclude_replies=False,
                    include_rts=False).items(limit=count_no) # 取得件数を指定

        index = 1

        for result in results:
            if 'media' in result.entities:
                image_url = result.extended_entities['media'][0]['media_url']
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]

                print(tweet)

                img_response = requests.get(image_url)
                match img_response.status_code:
                    case 200:
                        os.makedirs(f'{self.__username}', mode=0o777, exist_ok=True)
                        with open(f'{self.__username}/{index}.jpg', 'wb') as image:
                            image.write(img_response.content)

                        tweet['tweet'].append({"text": tweet_text, 'img': image_url, 'img_file': f'{index}.jpg'})
                        index += 1

                with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                    json.dump(tweet, f, indent=4, ensure_ascii=False)

In [ ]:
t = Tweet_text('erika_1837')
t.text_image_get()

In [ ]:
load = json.load(open('erika_1837.json', 'r'))
l = load['tweet']


for i in l:
    print(i['img'])
    img_response = requests.get(i['img'])

NG エロアカウントからテキストだけもらう

In [1]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力
        # テキストだけ
        target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])
        print(target_tweets)

        while True:
            next_token = target_tweets.meta['next_token']

            for tweets in target_tweets.data:
                text = tweets.data['text']
                tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])

            if next_token is None:
                break

        # if next_token:
        #     while True:
        #         target_tweets_next = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'], pagination_token=next_token)
        #         next_token = target_tweets_next.meta['next_token']

        #         for tweets in target_tweets_next.data:
        #             text = tweets.data['text']
        #             print(text)

        #             tweet['tweet'].append({"text": text})
        #             print(tweet)

        #             with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
        #                 json.dump(tweet, f, indent=4, ensure_ascii=False)

        #         if next_token is None:
        #             break

        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


In [ ]:
t = Tweet_text('hikarinspecial')
t.text_get

# テキストだけ取得（paginatorを使う）

In [1]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['retweets', 'replies'],max_results=100).flatten(limit=1000):

            text = tweets.data['text']
            tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)


In [2]:
t = Tweet_text('Lillian46612363')
t.text_get()

KeyboardInterrupt: 

# mizugazo更新分だけもらう

In [ ]:
old_json = json.load(open('old_mizugazo_all_photos.json', 'r'))

: 

# ツイ消し

In [2]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['replies'],max_results=100).flatten(limit=3000):

            self.client.delete_tweet(tweets.id)

            # JSON保存
            # text = tweets.data['text']
            # tweet['tweet'].append({"text": text})
            # with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
            #     json.dump(tweet, f, indent=4, ensure_ascii=False)

t = Tweet_text('katudon')
t.text_get()

# 動画だけダウンロード

In [ ]:
# TODO スパンキング動画

import tweepy
import API_config_katudon
import json
import requests
import re
import os


class Tweet_text:

    def __init__(self, username):

        auth = tweepy.OAuthHandler(API_config_katudon.API_KEY, API_config_katudon.API_SECRET)
        auth.set_access_token(API_config_katudon.ACCESS_TOKEN, API_config_katudon.ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.__username = username

    def text_image_get(self):


        count_no = 200
        tweet: dict[dict] = {}
        tweet['tweet'] = []

        results: iter = tweepy.Cursor(self.api.user_timeline, # タイムラインの取得
                    id=self.__username, # 取得対象のユーザーを指定
                    include_entities=True, # 省略されたリンクを全て取得
                    tweet_mode='extended', # 省略されたツイートを全て取得
                    lang='ja',
                    exclude_replies=False,
                    include_rts=False).items(limit=count_no) # 取得件数を指定

        index = 1

        for result in results:
            if 'media' in result.entities:
                image_url = result.extended_entities['media'][0]['media_url']
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]

                print(tweet)

                img_response = requests.get(image_url)
                match img_response.status_code:
                    case 200:
                        os.makedirs(f'{self.__username}', mode=0o777, exist_ok=True)
                        with open(f'{self.__username}/{index}.mp4', 'wb') as image:
                            image.write(img_response.content)

                        tweet['tweet'].append({"text": tweet_text, 'img': image_url, 'img_file': f'{index}.jpg'})
                        index += 1

                with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                    json.dump(tweet, f, indent=4, ensure_ascii=False)

t = Tweet_text('cutethigh')
t.text_image_get()

# Twitter api V2 自動RTだけ

In [2]:
import tweepy
import API_config_1oku
import time
import random


# Twitter API v2対応
client = tweepy.Client(consumer_key=API_config_1oku.API_KEY, consumer_secret=API_config_1oku.API_SECRET, access_token=API_config_1oku.ACCESS_TOKEN, access_token_secret=API_config_1oku.ACCESS_TOKEN_SECRET, bearer_token=API_config_1oku.Bearer_token)

# 取得したい件数 = RTする件数
counts = 10

#ランダム待機時間
wait1 = random.random()
wait2 = random.randint(30,50)
waitx = round(wait1 + wait2,3)

get_id = client.get_user(username='ShytoshiKusama')
response =  client.get_users_tweets(get_id.data['id'], exclude=['retweets', 'replies'])
print(response)

for tweet in response.data:
    try:
        client.retweet(tweet.id) # 自動RT
        time.sleep(waitx)

        print('RT完了!!')
    except Exception as e:
        print(e)


Response(data=[<Tweet id=1527839086148468736 text='https://t.co/Y7RPMxEQmy'>, <Tweet id=1527812130870796291 text="Thanks for letting me push in silence for this week. Next week things will begin to get interesting. \n\nAlso seen some GREAT art from .@PlaysideStudios for our Shib CCG. Can't wait to share. Great work mates!">, <Tweet id=1527350736819933207 text='Thanks for the chat about everything #SHIB .@redbull ! Looking forward to speaking more in the future! https://t.co/ROh3RceFyS'>, <Tweet id=1525236841011564544 text="Over 250k impressions on this tweet. If it helped even one person I am eternally grateful to be at your service.\n\nI was told once by a beautiful butterfly that the hardest part isn't flight; it's breaking, and then climbing out of a chrysalis and waiting for your wings to dry. https://t.co/GpW6JfBFjA">, <Tweet id=1524610261805670401 text='A personal medium post: The Semicolon Tattoo\n https://t.co/dQhnw72e3I'>, <Tweet id=1524559207055474695 text='https://t.co/ntkIe

KeyboardInterrupt: 